In [11]:
import mlflow
import mlflow.sklearn
from time import time
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             hamming_loss, jaccard_score, confusion_matrix, roc_curve, auc)
import matplotlib.pyplot as plt
import numpy as np


ModuleNotFoundError: No module named 'mlflow.sklearn'

## Récupération des données via l'API

In [ ]:
import requests
import time
import csv

def fetch_questions(page, pagesize, key):
    url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "page": page,
        "pagesize": pagesize,
        "order": "desc",
        "sort": "votes",
        "site": "stackoverflow",
        "filter": "!9_bDDxJY5",  # Custom filter to include necessary fields
        "key": key  # Include your API key here
    }
    response = requests.get(url, params=params)
    if response.status_code == 502:
        retry_after = int(response.headers.get('Retry-After', 1))
        print(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
        time.sleep(retry_after)
        return fetch_questions(page, pagesize, key)
    response.raise_for_status()  # Raises HTTPError for bad responses
    return response.json()

def filter_questions(questions):
    filtered = []
    for question in questions:
        # Check for view count and minimum number of tags
        if (question['view_count'] > 0 and
                question['score'] > 0 and
                question.get('favorite_count', 0) > 0 and
                len(question['tags']) - len(replace(question['tags'], '<','')) >= 5:
            filtered.append({
                "title": question['title'],
                "body": question['body'],
                "link": question['link'],
                "view_count": question['view_count'],
                "score": question['score'],
                "tags": ','.join(question['tags'])
            })
    return filtered

def main():
    all_questions = []
    page = 1
    pagesize = 50  # API limit for each batch
    max_questions = 50000
    api_key = "rl_Vssw9iwuVvjuT8agzZnVhH9J3"  # Replace with your actual API key

    while len(all_questions) < max_questions:
        try:
            data = fetch_questions(page, pagesize, api_key)
        except requests.exceptions.HTTPError as err:
            print(f"HTTP error occurred: {err}")
            break

        filtered_questions = filter_questions(data['items'])
        all_questions.extend(filtered_questions)
        page += 1

        if 'has_more' not in data or not data['has_more']:
            break

        # Respect API rate limits with dynamic sleep
        retry_after = int(data.get('backoff', 1))
        print(f"Pausing for {retry_after} seconds to respect rate limits.")
        time.sleep(retry_after)

    # Truncate the list to the desired number of questions
    all_questions = all_questions[:max_questions]

    # Save the questions to a CSV file
    csv_file = 'filtered_questions.csv'
    csv_columns = ["title", "body", "link", "view_count", "score", "tags"]
    
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for question in all_questions:
            writer.writerow(question)

    print(f"Collected {len(all_questions)} questions and saved to {csv_file}")

if __name__ == "__main__":
    main()

client id : 29476  
client secret : f*qvwypgwfHuuUzVJ4abow((  
key : rl_Vssw9iwuVvjuT8agzZnVhH9J3  
code : bOaC6F5f5Ct57Bcd7j0gPA))  